In [ ]:
Broadcast variable - create copy of dimension table on each worker node and performs operations - so no shuffle is needed and 
performance is better - cached in each node of cluster.ideal for joining tables where one of it is tiny tables .
Only suitable for smaller tables.its sent to worker thru driver.so need to fit into driver memory
With no broadcast - data is shuffled across all worker nodes for any transformations/ actions and permanent hits


In [ ]:
Broadcast variables (read-only shared variable)


Broadcast variables are shared, read-only variables cached and accessible across all nodes in a cluster for use by tasks. 
Rather than transmitting this data with every task, PySpark employs efficient broadcast algorithms to distribute broadcast variables
to machines, thereby reducing communication costs.broadcast variables are not sent to executors with sc.Broadcast(variable) call instead,
they will be sent to executors when they are first used.

Accumulator variables (updatable shared variables)
Accumulators are write-only and initialize once variables where only tasks that are running on workers are allowed to update and updates
from the workers get propagated automatically to the driver program. But, only the driver program is allowed to access the Accumulator 
variable using the value property.These variables are shared by all executors to update and add information through aggregation or 
computative operations



In [ ]:
# Create broadcast variable
broadcastVar = sc.broadcast([0, 1, 2, 3])
broadcastVar.value

In [ ]:
#Accumulator variables are created using SparkContext.longAccumulator(v)
# Create accumulator variable
accum = sc.longAccumulator("SumAccumulator")
sc.parallelize([1, 2, 3]).foreach(lambda x: accum.add(x))
#rdd.foreach() is executed on workers and accum.value is called from PySpark driver program.

In [ ]:
Transaction = [
    (100,'Cosmetic',150),
    (200,'Apparel',200),
    (300,'Shirt',300),
    (400,'Trouser',400),
    (500,'Socks',200),
    (100,'Belt',70),
    (200,'Cosmetic',200),
    (300,'Shoe',70),
    (400,'Socks',40),
    (500,'Shorts',100)
]

transaction_DF = spark.createDataFrame(data=Transaction,schema=['Store_id','Item','Amount'])
display(transaction_DF)

Store_id,Item,Amount
100,Cosmetic,150
200,Apparel,200
300,Shirt,300
400,Trouser,400
500,Socks,200
100,Belt,70
200,Cosmetic,200
300,Shoe,70
400,Socks,40
500,Shorts,100


In [ ]:
Store = [
    (100,'Store_London'),
    (200,'Store_Bangkok'),
    (300,'Store_UK'),
    (400,'Store_Amer'),
    (500,'Store_India')
]

store_DF = spark.createDataFrame(data= Store,schema=['Store_id','Store_name'])
display(store_DF)

Store_id,Store_name
100,Store_London
200,Store_Bangkok
300,Store_UK
400,Store_Amer
500,Store_India


In [ ]:
from pyspark.sql.functions import broadcast

join_DF = transaction_DF.join(broadcast(store_DF),transaction_DF['Store_id']== store_DF['Store_id'])
display(join_DF)

Store_id,Item,Amount,Store_id,Store_name
100,Cosmetic,150,100,Store_London
200,Apparel,200,200,Store_Bangkok
300,Shirt,300,300,Store_UK
400,Trouser,400,400,Store_Amer
500,Socks,200,500,Store_India
100,Belt,70,100,Store_London
200,Cosmetic,200,200,Store_Bangkok
300,Shoe,70,300,Store_UK
400,Socks,40,400,Store_Amer
500,Shorts,100,500,Store_India


In [ ]:
join_DF.explain(True)

== Parsed Logical Plan ==
Join Inner, (Store_id#36L = Store_id#15L)
:- LogicalRDD [Store_id#36L, Item#37, Amount#38L], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [Store_id#15L, Store_name#16], false

== Analyzed Logical Plan ==
Store_id: bigint, Item: string, Amount: bigint, Store_id: bigint, Store_name: string
Join Inner, (Store_id#36L = Store_id#15L)
:- LogicalRDD [Store_id#36L, Item#37, Amount#38L], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [Store_id#15L, Store_name#16], false

== Optimized Logical Plan ==
Join Inner, (Store_id#36L = Store_id#15L), rightHint=(strategy=broadcast)
:- Filter isnotnull(Store_id#36L)
:  +- LogicalRDD [Store_id#36L, Item#37, Amount#38L], false
+- Filter isnotnull(Store_id#15L)
   +- LogicalRDD [Store_id#15L, Store_name#16], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) BroadcastHashJoin [Store_id#36L], [Store_id#15L], Inner, BuildRight, false, true
   :- *(2) Filter isnotnull(S

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

rdd = spark.sparkContext.parallelize(data)

def state_convert(code):
    return broadcastStates.value[code]

result = rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)

[('James', 'Smith', 'USA', 'California'), ('Michael', 'Rose', 'USA', 'New York'), ('Robert', 'Williams', 'USA', 'California'), ('Maria', 'Jones', 'USA', 'Florida')]


In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

def state_convert(code):
    return broadcastStates.value[code]

result = df.rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).toDF(columns)
result.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |Williams|USA    |CA   |
|Maria    |Jones   |USA    |FL   |
+---------+--------+-------+-----+

+---------+--------+-------+----------+
|firstname|lastname|country|state     |
+---------+--------+-------+----------+
|James    |Smith   |USA    |California|
|Michael  |Rose    |USA    |New York  |
|Robert   |Williams|USA    |California|
|Maria    |Jones   |USA    |Florida   |
+---------+--------+-------+----------+



In [4]:
# Broadcast variable on filter
filteDf= df.where((df['state'].isin(broadcastStates.value)))


Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.functions.lit.
: org.apache.spark.SparkRuntimeException: The feature is not supported: literal for '{FL=Florida, NY=New York, CA=California}' of class java.util.HashMap.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.literalTypeUnsupportedError(QueryExecutionErrors.scala:296)
	at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:101)
	at org.apache.spark.sql.functions$.lit(functions.scala:125)
	at org.apache.spark.sql.functions.lit(functions.scala)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.Reflect

In [5]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("accumulator").getOrCreate()

accum=spark.sparkContext.accumulator(0)
rdd=spark.sparkContext.parallelize([1,2,3,4,5])
rdd.foreach(lambda x:accum.add(x))
print(accum.value)

accuSum=spark.sparkContext.accumulator(0)
def countFun(x):
    global accuSum
    accuSum+=x
rdd.foreach(countFun)
print(accuSum.value)

accumCount=spark.sparkContext.accumulator(0)
rdd2=spark.sparkContext.parallelize([1,2,3,4,5])
rdd2.foreach(lambda x:accumCount.add(1))
print(accumCount.value)

15
15
5
